In [1]:
# Clean output files
!rm -rf /kaggle/working/*

In [2]:
#|default_exp app

In [3]:
# Make sure we've got the latest version of fastai:
!pip install -Uqq fastai gradio nbdev

In [4]:
#|export
from fastai.vision.all import *
import gradio as gr

def is_cat(x): return x[0].isupper()

In [5]:
path_train = untar_data(URLs.PETS)/'images'

dls_train = ImageDataLoaders.from_name_func('.',
    get_image_files(path_train), valid_pct=0.2, seed=42,
    label_func=is_cat,
    item_tfms=Resize(192, method='squish'))

In [6]:
learn_train = vision_learner(dls_train, resnet18, metrics=error_rate)
learn_train.fine_tune(3)

In [7]:
learn_train.save('model.pkl')

In [8]:
im = PILImage.create('/kaggle/input/inputdata/dog.jpg')
im.thumbnail((192,192))
im

In [9]:
path_test = untar_data(URLs.PETS)/'images'

dls_test = ImageDataLoaders.from_name_func('.',
    get_image_files(path_test), valid_pct=0.2, seed=42,
    label_func=is_cat,
    item_tfms=Resize(192, method='squish'))

In [10]:
#|export
learn_test = vision_learner(dls_train, resnet18, metrics=error_rate)

In [11]:
#|export
learn_test.load('model.pkl')

In [12]:
learn_test.predict(im)

In [13]:
#|export
categories = ('Dog', 'Cat')

def classify_image(img):
    pred,idx,probs = learn_test.predict(img)
    return dict(zip(categories, map(float,probs)))

In [14]:
classify_image(im)

In [15]:
#|export
image = gr.Image()
label = gr.Label()
examples = ['dog.jpg', 'cat.jpg', 'dunno.jpg']

intf = gr.Interface(fn=classify_image, inputs=image, outputs=label, examples=examples)
intf.launch(inline=False)

In [16]:
from nbdev.export import nb_export